In [2]:
!nvcc --version
!pip install torchmd-net

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00

In [8]:
#!mkdir output
!CUDA_VISIBLE_DEVICES=0 torchmd-train --conf ET-QM9.yaml --log-dir output/

mkdir: cannot create directory ‘output’: File exists
Seed set to 1
train 110000, val 10000, test 10831
/usr/local/lib/python3.11/dist-packages/torchmdnet/priors/atomref.py:42: UserWarning: The atomref returned by the dataset is None, defaulting to zeros with max. atomic number 99. Maybe atomref is not defined for the current target.
  rank_zero_warn(
/usr/local/lib/python3.11/dist-packages/torchmdnet/scripts/train.py:249: UserWarning: WARNING: Test set will be evaluated every 10 epochs. This will slow down training.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory output/ exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
train 110000, val 10000, test 10831
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/callbacks/model_checkpoint.

In [ ]:
#test for inference 
import torch
import os
from torchmdnet.models.model import load_model

checkpoint = "/content/epoch=39-val_loss=0.0033-test_loss=0.0690.ckpt"
model = load_model(checkpoint, derivative=True)
# An arbitrary set of inputs for the model
n_atoms = 10
zs = torch.tensor([1, 6, 7, 8, 9], dtype=torch.long)
z = zs[torch.randint(0, len(zs), (n_atoms,))]
pos = torch.randn(len(z), 3)
batch = torch.zeros(len(z), dtype=torch.long)

y, neg_dy = model(z, pos, batch)

In [ ]:
def parse_qm9_data(data):
    lines = data.strip().split('\n')
    num_atoms = int(lines[0])  # First line gives the number of atoms
    atom_data = []

    for line in lines[1:]:
        parts = line.split()
        if len(parts) == 4:  # Atomic symbol and xyz positions
            atom_symbol = parts[0]
            x, y, z = map(float, parts[1:])
            atom_data.append((atom_symbol, x, y, z))

    return num_atoms, atom_data

def convert_to_tensor(atom_data):
    # Mapping of element symbols to atomic numbers (up to Z=32)
    element_to_atomic = {
        'H': 1, 'He': 2, 'Li': 3, 'Be': 4, 'B': 5, 'C': 6, 'N': 7, 'O': 8,
        'F': 9, 'Ne': 10, 'Na': 11, 'Mg': 12, 'Al': 13, 'Si': 14, 'P': 15,
        'S': 16, 'Cl': 17, 'Ar': 18, 'K': 19, 'Ca': 20, 'Sc': 21, 'Ti': 22,
        'V': 23, 'Cr': 24, 'Mn': 25, 'Fe': 26, 'Co': 27, 'Ni': 28, 'Cu': 29,
        'Zn': 30, 'Ga': 31, 'Ge': 32
    }

    atomic_numbers = []
    positions = []

    for symbol, x, y, z in atom_data:
        atomic_num = element_to_atomic[symbol]
        atomic_numbers.append(atomic_num)
        positions.append([x, y, z])

    positions_tensor = torch.tensor(positions, dtype=torch.float32)
    atomic_numbers = torch.tensor(atomic_numbers)
    return atomic_numbers, positions_tensor

In [ ]:
#running inference for one data point
folder_path = '/content/drive/MyDrive/structures_test'

path_list = [f'mol_{i}.xyz' for i in range(1, 5001)]

file = open(os.path.join(folder_path, path_list[0]), 'r')
data = file.read()
n, atom_data = parse_qm9_data(data)
z, pos = convert_to_tensor(atom_data)
batch = torch.zeros(len(z), dtype=torch.long)
y, neg_dy = model(z, pos, batch)
print(y)

In [ ]:
for p in path_list:
  file = open(os.path.join(folder_path, p), 'r')
  data = file.read()
  n, atom_data = parse_qm9_data(data)
  z, pos = convert_to_tensor(atom_data)
  batch = torch.zeros(len(z), dtype=torch.long)
  y, neg_dy = model(z, pos, batch)
  #write to csv file
  with open('output.csv', 'a') as f:
    f.write(f'{p[:-3]},{y.item()}\n')